In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import torch.autograd
import time
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
import dgl
from graphenvs import HalfCheetahGraphEnv
import itertools

Running on the GPU


Using backend: pytorch


In [3]:
class Network(nn.Module):
    def __init__(
        self,
        input_size,
        output_size,
        hidden_sizes,
        with_batch_norm=False,
        activation=None
    ):
        super(Network, self).__init__()
        self.hidden_sizes = hidden_sizes
        self.input_size = input_size
        self.output_size = output_size
        
        self.layers = nn.ModuleList()

        self.layers.append(nn.Linear(self.input_size, hidden_sizes[0]))
        if with_batch_norm:
            self.layers.append(nn.LayerNorm(normalized_shape=(hidden_sizes[0])))
        self.layers.append(nn.ReLU())
        
        for i in range(len(hidden_sizes) - 1):
            self.layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            if with_batch_norm:
                self.layers.append(nn.LayerNorm(normalized_shape=(hidden_sizes[i+1])))
            self.layers.append(nn.ReLU())
        
        self.layers.append(nn.Linear(hidden_sizes[len(hidden_sizes) - 1], self.output_size))
        
        if activation is not None:
            self.layers.append(activation())
            
    def forward(self, x):
        out = x
        
        for layer in self.layers:
            out = layer(out)
            
        return out


In [4]:
class GraphNeuralNetwork(nn.Module):
    def __init__(
        self,
        inputNetwork,
        messageNetwork,
        updateNetwork,
        outputNetwork,
        numMessagePassingIterations,
        withInputNetwork = True
    ):
        
        super(GraphNeuralNetwork, self).__init__()
                
        self.inputNetwork = inputNetwork
        self.messageNetwork = messageNetwork
        self.updateNetwork = updateNetwork
        self.outputNetwork = outputNetwork
        
        self.numMessagePassingIterations = numMessagePassingIterations
        self.withInputNetwork = withInputNetwork
        
    def inputFunction(self, nodes):
        return {'state' : self.inputNetwork(nodes.data['input'])}
    
    def messageFunction(self, edges):
        
        batchSize = edges.src['state'].shape[1]
        edgeData = edges.data['feature'].repeat(batchSize, 1).T.unsqueeze(-1)
        nodeInput = edges.src['input']
        
        return {'m' : self.messageNetwork(torch.cat((edges.src['state'], edgeData, nodeInput), -1))}
    
    def updateFunction(self, nodes):
        return {'state': self.updateNetwork(torch.cat((nodes.data['m_hat'], nodes.data['state']), -1))}
    
    def outputFunction(self, nodes):
        
        return {'output': self.outputNetwork(nodes.data['state'])}


    def forward(self, graph, state):
        
        self.update_states_in_graph(graph, state)
        
        if self.withInputNetwork:
            graph.apply_nodes(self.inputFunction)
        
        for messagePassingIteration in range(self.numMessagePassingIterations):
            graph.update_all(self.messageFunction, dgl.function.mean('m', 'm_hat'), self.updateFunction)
        
        graph.apply_nodes(self.outputFunction)
        
        output = graph.ndata['output']
        output = output.squeeze(-1).mean(0)
                
        return output
    
    def update_states_in_graph(self, graph, state):
        if len(state.shape) == 1:
            state = state.unsqueeze(0)
        
        numGraphFeature = 6
        numGlobalStateInformation = 5
        numLocalStateInformation = 2
        numStateVar = state.shape[1] // 2
        globalInformation = torch.cat((state[:, 0:5], state[:, numStateVar:numStateVar+5]), -1)
        
        numNodes = (numStateVar - 5) // 2

        nodeData = torch.empty((numNodes, state.shape[0], numGraphFeature + 2 * numGlobalStateInformation + 2 * numLocalStateInformation)).to(device)
        for nodeIdx in range(numNodes):

            # Assign global features from graph
            nodeData[nodeIdx, :, :6] = graph.ndata['feature'][nodeIdx]
            # Assign local state information
            nodeData[nodeIdx, :, 16] = state[:, 5 + nodeIdx]
            nodeData[nodeIdx, :, 17] = state[:, 5 + numNodes + nodeIdx]
            nodeData[nodeIdx, :, 18] = state[:, numStateVar + 5 + nodeIdx]
            nodeData[nodeIdx, :, 19] = state[:, numStateVar + 5 + numNodes + nodeIdx]

        # Assdign global state information
        nodeData[:, :, 6:16] = globalInformation
        
        if self.withInputNetwork:
            graph.ndata['input'] = nodeData        
        
        else:
            graph.ndata['state'] = nodeData


In [5]:
trainingIdxs = [0, 1, 2, 3, 4, 5]

In [6]:
states = {}
actions = {}
rewards = {}
next_states = {}
dones = {}
env = {}

for morphIdx in trainingIdxs:

    prefix = '../datasets/{}/'.format(morphIdx)
    
    states[morphIdx] = np.load(prefix + 'states_array.npy')
    actions[morphIdx] = np.load(prefix + 'actions_array.npy')
    rewards[morphIdx] = np.load(prefix + 'rewards_array.npy')
    next_states[morphIdx] = np.load(prefix + 'next_states_array.npy')
    dones[morphIdx] = np.load(prefix + 'dones_array.npy')
    
    env[morphIdx] = HalfCheetahGraphEnv(None)
    env[morphIdx].set_morphology(morphIdx)

/home/ovi/anaconda3/envs/honors-project/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
NoneType: None


None
*************************************************************************************************************


NoneType: None


None
*************************************************************************************************************


NoneType: None


...

NoneType: None


None
*************************************************************************************************************


NoneType: None


None
*************************************************************************************************************
None
*************************************************************************************************************


NoneType: None


In [7]:
states_train = {}
states_test = {}
next_states_train = {}
next_states_test = {}

for morphIdx in trainingIdxs:
    permutation = np.random.permutation(states[morphIdx].shape[0])
    
    states[morphIdx] = states[morphIdx][permutation]
    next_states[morphIdx] = next_states[morphIdx][permutation]
    
    states_train[morphIdx] = torch.from_numpy(states[morphIdx][100000:]).float()
    states_test[morphIdx] = torch.from_numpy(states[morphIdx][:100000]).float()
    
    next_states_train[morphIdx] = torch.from_numpy(next_states[morphIdx][100000:]).float()
    next_states_test[morphIdx] = torch.from_numpy(next_states[morphIdx][:100000]).float()

In [8]:
hidden_sizes = [256, 256]

inputSize = 20
stateSize = 64
messageSize = 64
outputSize = 1
numMessagePassingIterations = 6
batch_size = 2048
with_batch_norm=True
numBatchesPerTrainingStep = 1

inputNetwork = Network(inputSize, stateSize, hidden_sizes, with_batch_norm)
messageNetwork = Network(stateSize + inputSize + 1, messageSize, hidden_sizes, with_batch_norm, nn.Tanh)
updateNetwork = Network(stateSize + messageSize, stateSize, hidden_sizes, with_batch_norm)
outputNetwork = Network(stateSize, outputSize, hidden_sizes, with_batch_norm, nn.Sigmoid)

gnn = GraphNeuralNetwork(inputNetwork, messageNetwork, updateNetwork, outputNetwork, numMessagePassingIterations).to(device)

In [10]:
lr = 1e-5
optimizer = optim.Adam(itertools.chain(inputNetwork.parameters(), messageNetwork.parameters(), updateNetwork.parameters(), outputNetwork.parameters())
                       , lr=lr, weight_decay=0)

# lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0, verbose=True, min_lr=1e-5)
binaryLoss = nn.BCELoss()

zeroTensor = torch.zeros([batch_size]).to(device)
oneTensor = torch.ones([batch_size]).to(device)

In [11]:
numTrainingBatches = int(np.ceil(states_train[trainingIdxs[0]].shape[0] / batch_size))
numTestingBatches = int(np.ceil(states_test[trainingIdxs[0]].shape[0] / batch_size))

trainLosses = {}
testLosses = {}

for morphIdx in trainingIdxs:
    trainLosses[morphIdx] = []
    testLosses[morphIdx] = []

In [12]:
for epoch in range(15):
    
    print('Starting Epoch {}'.format(epoch))
    # Record epoch start time to calculate per epoch time
    epoch_t0 = time.time()
    
    # Randomize the order of traininig examples
    for morphIdx in trainingIdxs:
        permutation = np.random.permutation(states_train[morphIdx].shape[0])

        states_train[morphIdx] = states_train[morphIdx][permutation]
        next_states_train[morphIdx] = next_states_train[morphIdx][permutation]

    with torch.no_grad():
        
        for morphIdx in trainingIdxs:
        
            testLosses[morphIdx].append(torch.zeros(4))
            
            for batch_ in range(0, numTestingBatches-1):
                
                # Get new graphs for each iteration
                g1 = env[morphIdx].get_graph()._get_dgl_graph()
                g2 = env[morphIdx].get_graph()._get_dgl_graph()
                
                current_states = states_test[morphIdx][batch_ * batch_size:(batch_+1)*batch_size]
#                 forward_states = next_states_test[morphIdx][batch_ * batch_size:(batch_+1)*batch_size]
                forward_states = current_states - next_states_test[morphIdx][batch_ * batch_size:(batch_+1)*batch_size]

                forward_x = torch.cat((current_states, forward_states), -1).to(device)
                predicted_sigmoids = gnn(g1, forward_x)
                forwardLoss = binaryLoss(predicted_sigmoids, oneTensor)
                
                # Save Forward Loss and Accuracy
                testLosses[morphIdx][-1][0] += forwardLoss.item()
                testLosses[morphIdx][-1][1] += torch.eq(oneTensor, torch.round(predicted_sigmoids)).sum().item() / float(batch_size)

                choices_range = np.arange(states_test[morphIdx].shape[0])
                random_indeces = np.random.choice(choices_range, size=current_states.shape[0])
                
#                 random_states = states_test[morphIdx][random_indeces]
                random_states = current_states - states_test[morphIdx][random_indeces]

                random_x = torch.cat((current_states, random_states), -1).to(device)
                
                predicted_sigmoids = gnn(g2, random_x)

                randomLoss = binaryLoss(predicted_sigmoids, zeroTensor)

                testLosses[morphIdx][-1][2] += randomLoss.item()
                testLosses[morphIdx][-1][3] += torch.eq(zeroTensor, torch.round(predicted_sigmoids)).sum().item() / float(batch_size)
            testLosses[morphIdx][-1] /= numTestingBatches
    
    for morphIdx in trainingIdxs:
        print('Test Idx {} | F-Loss {:.3f} | F-Acc {:.3f} | R-Loss {:.3f} | R-Acc {:.3f}'.
            format(morphIdx, testLosses[morphIdx][-1][0], testLosses[morphIdx][-1][1], testLosses[morphIdx][-1][2], testLosses[morphIdx][-1][3]))

    
    for batch in range(0, numTrainingBatches-1, numBatchesPerTrainingStep):
                
        batch_t0 = time.time()
        
        for morphIdx in trainingIdxs:
            trainLosses[morphIdx].append(torch.zeros(4))

        optimizer.zero_grad()
        
        for batchOffset in range(numBatchesPerTrainingStep):

            if batch + batchOffset >= numTrainingBatches - 1:
                break
                
            for morphIdx in trainingIdxs:
                
                # Get new graphs for each iteration
                g1 = env[morphIdx].get_graph()._get_dgl_graph()
                g2 = env[morphIdx].get_graph()._get_dgl_graph()
                
                current_states = states_train[morphIdx][(batch + batchOffset) * batch_size:(batch + batchOffset + 1)*batch_size]
#                 forward_states = next_states_train[morphIdx][(batch + batchOffset) * batch_size:(batch + batchOffset + 1)*batch_size]
                forward_states = current_states - next_states_train[morphIdx][(batch + batchOffset) * batch_size:(batch + batchOffset + 1)*batch_size]

                forward_x = torch.cat((current_states, forward_states), -1).to(device)
                
                predicted_sigmoids = gnn(g1, forward_x)
                forwardLoss = binaryLoss(predicted_sigmoids, oneTensor)
                
                # Save Forward Loss and Accuracy
                trainLosses[morphIdx][-1][0] += forwardLoss.item()
                trainLosses[morphIdx][-1][1] += torch.eq(oneTensor, torch.round(predicted_sigmoids)).sum().item() / float(batch_size)

                forwardLoss.backward()


                choices_range = np.arange(states_train[morphIdx].shape[0])
                random_indeces = np.random.choice(choices_range, size=batch_size)

#                 random_states = states_train[morphIdx][random_indeces]
                random_states = current_states - states_train[morphIdx][random_indeces]
                
                random_x = torch.cat((current_states, random_states), -1).to(device)
                
                predicted_sigmoids = gnn(g2, random_x)

                randomLoss = binaryLoss(predicted_sigmoids, zeroTensor)
                randomLoss.backward()

                trainLosses[morphIdx][-1][2] += randomLoss.item()
                trainLosses[morphIdx][-1][3] += torch.eq(zeroTensor, torch.round(predicted_sigmoids)).sum().item() / float(batch_size)
                        
        for morphIdx in trainingIdxs:
            trainLosses[morphIdx][-1] /= numBatchesPerTrainingStep

        optimizer.step()
        batch_time = time.time() - batch_t0

        if batch % 200 == 0:
            print('Batch {} in {:.2f}s'.format(batch, batch_time))
            
            for morphIdx in trainingIdxs:
                print('Train Idx {} | F-Loss {:.3f} | F-Acc {:.3f}| R-Loss {:.3f} | R-Acc {:.3f}'.
                    format(morphIdx, trainLosses[morphIdx][-1][0], trainLosses[morphIdx][-1][1], trainLosses[morphIdx][-1][2], trainLosses[morphIdx][-1][3]))


    print('Epoch {} finished in {:.1f}s'.format(epoch, time.time() - epoch_t0))


Starting Epoch 0
Test Idx 0 | F-Loss 0.870 | F-Acc 0.000 | R-Loss 0.516 | R-Acc 0.980
Test Idx 1 | F-Loss 0.861 | F-Acc 0.000 | R-Loss 0.519 | R-Acc 0.980
Test Idx 2 | F-Loss 0.870 | F-Acc 0.000 | R-Loss 0.516 | R-Acc 0.980
Test Idx 3 | F-Loss 0.863 | F-Acc 0.000 | R-Loss 0.519 | R-Acc 0.980
Test Idx 4 | F-Loss 0.864 | F-Acc 0.000 | R-Loss 0.526 | R-Acc 0.980
Test Idx 5 | F-Loss 0.858 | F-Acc 0.000 | R-Loss 0.526 | R-Acc 0.980
Batch 0 in 1.95s
Train Idx 0 | F-Loss 0.889 | F-Acc 0.000| R-Loss 0.527 | R-Acc 1.000
Train Idx 1 | F-Loss 0.879 | F-Acc 0.000| R-Loss 0.531 | R-Acc 1.000
Train Idx 2 | F-Loss 0.887 | F-Acc 0.000| R-Loss 0.526 | R-Acc 1.000
Train Idx 3 | F-Loss 0.882 | F-Acc 0.000| R-Loss 0.531 | R-Acc 1.000
Train Idx 4 | F-Loss 0.882 | F-Acc 0.000| R-Loss 0.537 | R-Acc 1.000
Train Idx 5 | F-Loss 0.874 | F-Acc 0.000| R-Loss 0.538 | R-Acc 1.000
Batch 200 in 2.36s
Train Idx 0 | F-Loss 0.199 | F-Acc 0.953| R-Loss 0.255 | R-Acc 0.897
Train Idx 1 | F-Loss 0.157 | F-Acc 0.963| R-Loss 0

Batch 400 in 2.00s
Train Idx 0 | F-Loss 0.038 | F-Acc 0.990| R-Loss 0.068 | R-Acc 0.981
Train Idx 1 | F-Loss 0.030 | F-Acc 0.992| R-Loss 0.025 | R-Acc 0.993
Train Idx 2 | F-Loss 0.029 | F-Acc 0.992| R-Loss 0.026 | R-Acc 0.993
Train Idx 3 | F-Loss 0.022 | F-Acc 0.996| R-Loss 0.025 | R-Acc 0.993
Train Idx 4 | F-Loss 0.037 | F-Acc 0.988| R-Loss 0.054 | R-Acc 0.983
Train Idx 5 | F-Loss 0.032 | F-Acc 0.992| R-Loss 0.038 | R-Acc 0.989
Epoch 4 finished in 921.9s
Starting Epoch 5
Test Idx 0 | F-Loss 0.025 | F-Acc 0.973 | R-Loss 0.061 | R-Acc 0.960
Test Idx 1 | F-Loss 0.017 | F-Acc 0.975 | R-Loss 0.033 | R-Acc 0.970
Test Idx 2 | F-Loss 0.018 | F-Acc 0.975 | R-Loss 0.038 | R-Acc 0.968
Test Idx 3 | F-Loss 0.017 | F-Acc 0.976 | R-Loss 0.037 | R-Acc 0.968
Test Idx 4 | F-Loss 0.023 | F-Acc 0.975 | R-Loss 0.053 | R-Acc 0.963
Test Idx 5 | F-Loss 0.021 | F-Acc 0.976 | R-Loss 0.038 | R-Acc 0.967
Batch 0 in 2.01s
Train Idx 0 | F-Loss 0.030 | F-Acc 0.993| R-Loss 0.051 | R-Acc 0.982
Train Idx 1 | F-Loss 0.

Batch 200 in 1.96s
Train Idx 0 | F-Loss 0.017 | F-Acc 0.996| R-Loss 0.040 | R-Acc 0.987
Train Idx 1 | F-Loss 0.008 | F-Acc 0.999| R-Loss 0.019 | R-Acc 0.994
Train Idx 2 | F-Loss 0.011 | F-Acc 0.998| R-Loss 0.023 | R-Acc 0.993
Train Idx 3 | F-Loss 0.010 | F-Acc 0.998| R-Loss 0.027 | R-Acc 0.993
Train Idx 4 | F-Loss 0.015 | F-Acc 0.998| R-Loss 0.027 | R-Acc 0.989
Train Idx 5 | F-Loss 0.013 | F-Acc 0.998| R-Loss 0.030 | R-Acc 0.990
Batch 400 in 1.96s
Train Idx 0 | F-Loss 0.020 | F-Acc 0.997| R-Loss 0.034 | R-Acc 0.990
Train Idx 1 | F-Loss 0.013 | F-Acc 0.998| R-Loss 0.028 | R-Acc 0.993
Train Idx 2 | F-Loss 0.010 | F-Acc 0.998| R-Loss 0.015 | R-Acc 0.996
Train Idx 3 | F-Loss 0.012 | F-Acc 0.995| R-Loss 0.024 | R-Acc 0.994
Train Idx 4 | F-Loss 0.020 | F-Acc 0.995| R-Loss 0.035 | R-Acc 0.991
Train Idx 5 | F-Loss 0.013 | F-Acc 0.997| R-Loss 0.026 | R-Acc 0.991
Epoch 9 finished in 902.9s
Starting Epoch 10
Test Idx 0 | F-Loss 0.017 | F-Acc 0.975 | R-Loss 0.032 | R-Acc 0.970
Test Idx 1 | F-Loss 

Batch 0 in 1.99s
Train Idx 0 | F-Loss 0.009 | F-Acc 0.998| R-Loss 0.030 | R-Acc 0.993
Train Idx 1 | F-Loss 0.014 | F-Acc 0.997| R-Loss 0.013 | R-Acc 0.996
Train Idx 2 | F-Loss 0.011 | F-Acc 0.998| R-Loss 0.012 | R-Acc 0.998
Train Idx 3 | F-Loss 0.007 | F-Acc 0.998| R-Loss 0.020 | R-Acc 0.996
Train Idx 4 | F-Loss 0.013 | F-Acc 0.995| R-Loss 0.026 | R-Acc 0.991
Train Idx 5 | F-Loss 0.013 | F-Acc 0.995| R-Loss 0.011 | R-Acc 0.997
Batch 200 in 1.96s
Train Idx 0 | F-Loss 0.012 | F-Acc 0.997| R-Loss 0.026 | R-Acc 0.994
Train Idx 1 | F-Loss 0.006 | F-Acc 0.999| R-Loss 0.014 | R-Acc 0.994
Train Idx 2 | F-Loss 0.017 | F-Acc 0.996| R-Loss 0.012 | R-Acc 0.995
Train Idx 3 | F-Loss 0.012 | F-Acc 0.997| R-Loss 0.005 | R-Acc 0.999
Train Idx 4 | F-Loss 0.011 | F-Acc 0.998| R-Loss 0.025 | R-Acc 0.991
Train Idx 5 | F-Loss 0.016 | F-Acc 0.994| R-Loss 0.010 | R-Acc 0.997
Batch 400 in 1.97s
Train Idx 0 | F-Loss 0.013 | F-Acc 0.996| R-Loss 0.014 | R-Acc 0.995
Train Idx 1 | F-Loss 0.008 | F-Acc 0.998| R-Loss

In [13]:
torch.save(gnn.state_dict(), 'mixed-delta-validTransition.pt')

In [9]:
gnn.load_state_dict(torch.load('mixed-delta-validTransition.pt'))

<All keys matched successfully>

In [14]:
backwardLosses = {}

for morphIdx in trainingIdxs:
    
    backwardLosses[morphIdx] = np.zeros(2)
        
    with torch.no_grad():
        
        testLosses[morphIdx].append(torch.zeros(4))

        for batch_ in range(0, numTestingBatches-1):

            # Get new graphs for each iteration
            g1 = env[morphIdx].get_graph()._get_dgl_graph()

            current_states = states_test[morphIdx][batch_ * batch_size:(batch_+1)*batch_size]
            forward_states = next_states_test[morphIdx][batch_ * batch_size:(batch_+1)*batch_size]
            backward_x = torch.cat((forward_states, forward_states - current_states), -1).to(device)
            predicted_sigmoids = gnn(g1, backward_x)
            backward_loss = binaryLoss(predicted_sigmoids, zeroTensor)

            # Save Forward Loss and Accuracy
            backwardLosses[morphIdx][0] += backward_loss.item()
            backwardLosses[morphIdx][1] += torch.eq(zeroTensor, torch.round(predicted_sigmoids)).sum().item() / float(batch_size)

        backwardLosses[morphIdx] /= numTestingBatches
            
print(backwardLosses)

{0: array([0.62110414, 0.82023278]), 1: array([0.03034668, 0.97377232]), 2: array([0.02481634, 0.97454959]), 3: array([0.04841778, 0.97093232]), 4: array([0.06627391, 0.96639828]), 5: array([0.06741389, 0.96696628])}


In [20]:
velocityChangeLosses = {}
num_samples = 2048
num_states = 100

for morphIdx in trainingIdxs:
    velocityChangeLosses[morphIdx] = np.zeros(2)
        
    with torch.no_grad():
        
                    
        state_indeces = np.random.choice(np.arange(states_test[morphIdx].shape[0]), size=num_states)

        for state_idx in state_indeces:

            # Get new graphs for each iteration
            g1 = env[morphIdx].get_graph()._get_dgl_graph()

            current_states = states_test[morphIdx][state_idx].repeat(num_samples, 1)
            velocities = torch.from_numpy(np.linspace(start=0, stop=2, num=num_samples))
            forward_states = current_states
            forward_states[:, 0] = velocities
            velocities_changed_x = torch.cat((current_states, current_states - forward_states), -1).to(device)
            predicted_sigmoids = gnn(g1, velocities_changed_x)
            velocity_changed_loss = binaryLoss(predicted_sigmoids, zeroTensor)
            
#             print(predicted_sigmoids)
            
            # Save Forward Loss and Accuracy
            velocityChangeLosses[morphIdx][0] += velocity_changed_loss.item()
            velocityChangeLosses[morphIdx][1] += torch.eq(zeroTensor, torch.round(predicted_sigmoids)).sum().item() / float(batch_size)

        velocityChangeLosses[morphIdx] /= num_states
            
print(velocityChangeLosses)

{0: array([2.23183527, 0.54564941]), 1: array([1.36457576, 0.71598633]), 2: array([0.65920722, 0.8647168 ]), 3: array([1.41406122, 0.72076172]), 4: array([0.71103541, 0.84983398]), 5: array([1.23965704, 0.70364258])}
